# Recommender Systems 2021/22

### Practice - FunkSVD implemented with Python and Cython

FunkSVD is one of the simplest and most known matrix factorization models for rating prediction. It was proposed by Simon Funk in a now famous post on his website (with a cow grazing around Auckland): https://sifter.org/~simon/journal/20061211.html

In [1]:
import time
import numpy as np

In [2]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

data_reader = Movielens10MReader()
data_loaded = data_reader.load_data()

URM_all = data_loaded.get_URM_all()

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

Movielens10M: Verifying data consistency...
Movielens10M: Verifying data consistency... Passed!
DataReader: current dataset is: <class 'Data_manager.Dataset.Dataset'>
	Number of items: 10681
	Number of users: 69878
	Number of interactions in URM_all: 10000054
	Value range in URM_all: 0.50-5.00
	Interaction density: 1.34E-02
	Interactions per user:
		 Min: 2.00E+01
		 Avg: 1.43E+02
		 Max: 7.36E+03
	Interactions per item:
		 Min: 0.00E+00
		 Avg: 9.36E+02
		 Max: 3.49E+04
	Gini Index: 0.57

	ICM name: ICM_genres, Value range: 1.00 / 1.00, Num features: 20, feature occurrences: 21564, density 1.01E-01
	ICM name: ICM_tags, Value range: 1.00 / 69.00, Num features: 10217, feature occurrences: 108563, density 9.95E-04
	ICM name: ICM_all, Value range: 1.00 / 69.00, Num features: 10237, feature occurrences: 130127, density 1.19E-03




In [3]:
URM_train

<69878x10681 sparse matrix of type '<class 'numpy.float64'>'
	with 8000088 stored elements in Compressed Sparse Row format>

### What do we need for FunkSVD?

* User factor and Item factor matrices
* Computing prediction
* Update rule
* Training loop and some patience


In [4]:
n_users, n_items = URM_train.shape

## Step 1: We create the dense latent factor matrices
### In a MF model you have two matrices, one with a row per user and the other with a column per item. The other dimension, columns for the first one and rows for the second one is called latent factors

In [5]:
num_factors = 10

user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

In [6]:
user_factors

array([[0.10922357, 0.1400291 , 0.83540861, ..., 0.00427209, 0.61478403,
        0.72202759],
       [0.79784576, 0.1905475 , 0.76621221, ..., 0.72553039, 0.18384262,
        0.19075107],
       [0.0112557 , 0.38364501, 0.14426158, ..., 0.61721718, 0.48454021,
        0.04158188],
       ...,
       [0.73182207, 0.18786462, 0.22171543, ..., 0.06294801, 0.18048496,
        0.10350385],
       [0.21114588, 0.40653902, 0.90383763, ..., 0.40786238, 0.50300933,
        0.38190921],
       [0.81091948, 0.18710044, 0.64201128, ..., 0.95273574, 0.76063279,
        0.3603066 ]])

In [7]:
item_factors

array([[0.497202  , 0.23804075, 0.76369035, ..., 0.70437787, 0.23605717,
        0.61427761],
       [0.00393812, 0.67987897, 0.68433483, ..., 0.69707505, 0.2039948 ,
        0.22130279],
       [0.54465904, 0.9514584 , 0.75404424, ..., 0.04597385, 0.99216349,
        0.1186468 ],
       ...,
       [0.09989421, 0.07621387, 0.55361184, ..., 0.4095528 , 0.31555305,
        0.70204314],
       [0.44115908, 0.29720619, 0.08158752, ..., 0.96194354, 0.77475201,
        0.03910538],
       [0.85648627, 0.15503346, 0.67365132, ..., 0.20409353, 0.02832431,
        0.30959996]])

## Step 2: We sample an interaction and compute the prediction of the current FunkSVD model

In [8]:
URM_train_coo = URM_train.tocoo()

sample_index = np.random.randint(URM_train_coo.nnz)
sample_index

1929120

In [9]:
user_id = URM_train_coo.row[sample_index]
item_id = URM_train_coo.col[sample_index]
rating = URM_train_coo.data[sample_index]

(user_id, item_id, rating)

(17025, 468, 3.0)

In [10]:
predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])
predicted_rating

1.969457703505743

#### The first predicted rating is a random prediction, essentially

### Step 3: We compute the prediction error and update the latent factor matrices

In [11]:
prediction_error = rating - predicted_rating
prediction_error

1.030542296494257

### The error is positive, so we need to increase the prediction our model computes. Meaning, we have to increase the values latent factor matrices

### Which latent factors we modify? All the factors of the item and user we used

In [12]:
# Copy original value to avoid messing up the updates
H_i = item_factors[item_id,:]
W_u = user_factors[user_id,:]

In [13]:
H_i

array([0.07456019, 0.62647858, 0.13277896, 0.84962947, 0.00383289,
       0.63947138, 0.74936389, 0.64903736, 0.57800548, 0.45869254])

In [14]:
W_u

array([0.95626918, 0.53530823, 0.07668639, 0.36144233, 0.0690484 ,
       0.8392761 , 0.38958578, 0.27577824, 0.25139705, 0.20128098])

#### Apply the update rule

In [15]:
learning_rate = 1e-4
regularization = 1e-5

In [16]:
user_factors_update = prediction_error * H_i - regularization * W_u
user_factors_update

array([0.07682787, 0.64560732, 0.13683357, 0.87557549, 0.00394927,
       0.65899391, 0.77224729, 0.6688577 , 0.59565659, 0.47270005])

In [17]:
item_factors_update = prediction_error * W_u - regularization * H_i
item_factors_update

array([0.98547509, 0.5516515 , 0.07902724, 0.37247312, 0.07115726,
       0.86490313, 0.40147713, 0.28419465, 0.25906952, 0.20742397])

In [18]:
user_factors[user_id,:] += learning_rate * user_factors_update 
item_factors[item_id,:] += learning_rate * item_factors_update

### Let's check what the new prediction for the same user-item interaction would be

In [19]:
predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])
predicted_rating

1.9700195703562

### The value is slightly higher than before, we are moving in the right direction

### And now? Sample another interaction and repeat... a lot of times

### WARNING: Initialization must be done with random non-zero values ... otherwise

In [20]:
user_factors = np.zeros((n_users, num_factors))
item_factors = np.zeros((n_items, num_factors))

predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])

print("Prediction is {:.2f}".format(predicted_rating))

prediction_error = rating - predicted_rating

print("Prediction error is {:.2f}".format(prediction_error))


Prediction is 0.00
Prediction error is 3.00


In [21]:
H_i = item_factors[item_id,:]
W_u = user_factors[user_id,:]

user_factors[user_id,:] += learning_rate * (prediction_error * H_i - regularization * W_u)
item_factors[item_id,:] += learning_rate * (prediction_error * W_u - regularization * H_i)

In [22]:
predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])

print("Prediction after the update is {:.2f}".format(predicted_rating))
print("Prediction error is {:.2f}".format(rating - predicted_rating))

Prediction after the update is 0.00
Prediction error is 3.00


### Since the matrices are multiplied, if we initialize one of them as zero, the updates will always be zero and the model will not be able to learn.

### Let's put all together in a training loop.

In [23]:
URM_train_coo = URM_train.tocoo()

num_factors = 10
learning_rate = 1e-4

user_factors = np.random.random((n_users, num_factors))
item_factors = np.random.random((n_items, num_factors))

loss = 0.0
start_time = time.time()

for sample_num in range(1000000):
    
    # Randomly pick sample
    sample_index = np.random.randint(URM_train_coo.nnz)

    user_id = URM_train_coo.row[sample_index]
    item_id = URM_train_coo.col[sample_index]
    rating = URM_train_coo.data[sample_index]

    # Compute prediction
    predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])
        
    # Compute prediction error, or gradient
    prediction_error = rating - predicted_rating
    loss += prediction_error**2
    
    # Copy original value to avoid messing up the updates
    H_i = item_factors[item_id,:]
    W_u = user_factors[user_id,:]  
    
    user_factors_update = prediction_error * H_i - regularization * W_u
    item_factors_update = prediction_error * W_u - regularization * H_i
    
    user_factors[user_id,:] += learning_rate * user_factors_update 
    item_factors[item_id,:] += learning_rate * item_factors_update    
    
    # Print some stats
    if (sample_num +1)% 100000 == 0:
        elapsed_time = time.time() - start_time
        samples_per_second = sample_num/elapsed_time
        print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

Iteration 100000 in 1.56 seconds, loss is 2.64. Samples per second 64263.03
Iteration 200000 in 3.16 seconds, loss is 2.61. Samples per second 63247.17
Iteration 300000 in 4.75 seconds, loss is 2.57. Samples per second 63140.78
Iteration 400000 in 6.32 seconds, loss is 2.55. Samples per second 63337.47
Iteration 500000 in 7.89 seconds, loss is 2.53. Samples per second 63359.57
Iteration 600000 in 9.46 seconds, loss is 2.50. Samples per second 63394.41
Iteration 700000 in 11.03 seconds, loss is 2.48. Samples per second 63436.56
Iteration 800000 in 12.60 seconds, loss is 2.46. Samples per second 63473.24
Iteration 900000 in 14.16 seconds, loss is 2.44. Samples per second 63542.15
Iteration 1000000 in 15.74 seconds, loss is 2.42. Samples per second 63540.81


### What do we see? The loss oscillates over time, sometimes it goes down sometimes up.
### How long do we train such a model?

* An epoch: a complete loop over all the train data
* Usually you train for multiple epochs. Depending on the algorithm and data 10s or 100s of epochs.

In [24]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 1259.03 seconds, or 20.98 minutes


### This model is relatively quick, less than 30 minutes

### Let's see what we can do with Cython
### First step, just compile it. We do not have the data at compile time, so we put the loop in a function

In [25]:
%load_ext Cython

In [26]:
%%cython
import numpy as np
import time

def do_some_training(URM_train):

    URM_train_coo = URM_train.tocoo()
    n_users, n_items = URM_train_coo.shape

    num_factors = 10
    learning_rate = 1e-4
    regularization = 1e-5

    user_factors = np.random.random((n_users, num_factors))
    item_factors = np.random.random((n_items, num_factors))

    loss = 0.0
    start_time = time.time()

    for sample_num in range(1000000):

        # Randomly pick sample
        sample_index = np.random.randint(URM_train_coo.nnz)

        user_id = URM_train_coo.row[sample_index]
        item_id = URM_train_coo.col[sample_index]
        rating = URM_train_coo.data[sample_index]

        # Compute prediction
        predicted_rating = np.dot(user_factors[user_id,:], item_factors[item_id,:])

        # Compute prediction error, or gradient
        prediction_error = rating - predicted_rating
        loss += prediction_error**2

        # Copy original value to avoid messing up the updates
        H_i = item_factors[item_id,:]
        W_u = user_factors[user_id,:]  

        user_factors_update = prediction_error * H_i - regularization * W_u
        item_factors_update = prediction_error * W_u - regularization * H_i

        user_factors[user_id,:] += learning_rate * user_factors_update 
        item_factors[item_id,:] += learning_rate * item_factors_update    

        # Print some stats
        if (sample_num +1)% 100000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = sample_num/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

    return loss, samples_per_second

In [27]:
loss, samples_per_second = do_some_training(URM_train)

Iteration 100000 in 1.38 seconds, loss is 2.61. Samples per second 72406.42
Iteration 200000 in 2.79 seconds, loss is 2.58. Samples per second 71757.29
Iteration 300000 in 4.19 seconds, loss is 2.56. Samples per second 71611.80
Iteration 400000 in 5.59 seconds, loss is 2.52. Samples per second 71513.70
Iteration 500000 in 7.04 seconds, loss is 2.50. Samples per second 70978.19
Iteration 600000 in 8.45 seconds, loss is 2.47. Samples per second 71035.36
Iteration 700000 in 9.86 seconds, loss is 2.45. Samples per second 70989.75
Iteration 800000 in 11.25 seconds, loss is 2.43. Samples per second 71138.57
Iteration 900000 in 12.64 seconds, loss is 2.41. Samples per second 71209.65
Iteration 1000000 in 14.06 seconds, loss is 2.39. Samples per second 71124.67


In [28]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 1124.79 seconds, or 18.75 minutes


### The compiler is just porting in C all operations that the python interpreter would have to perform, dynamic tiping included. Have a look at the html reports in the Cython_examples folder

### Now try to add some types: If you use a variable only as a C object, use primitive tipes

* cdef int namevar
* cdef double namevar
* cdef float namevar
* cdef double[:] singledimensionarray
* cdef double[:,:] bidimensionalmatrix

### Some operations are still done with sparse matrices, those cannot be correctly optimized because the compiler does not know how what is the type of the data.

### To address this, we create typed arrays in which we put the URM_train data
####  For example, this operation: user_id = URM_train_coo.row[sample_index]
#### Becomes:
#### cdef int user_id
#### cdef int[:] URM_train_coo_row = URM_train_coo.row
#### user_id = URM_train_coo_row[sample_index]

### We can also skip the creation of the items_in_user_profile array and replace the np.random call with the faster native C function rand()


### We now use types for all main variables


In [29]:
%%cython
import numpy as np
import time

from libc.stdlib cimport rand, srand, RAND_MAX

def do_some_training(URM_train):

    URM_train_coo = URM_train.tocoo()
    n_users, n_items = URM_train_coo.shape
    cdef int n_interactions = URM_train.nnz
    
    cdef int sample_num, sample_index, user_id, item_id, factor_index
    cdef double rating, predicted_rating, prediction_error

    cdef int num_factors = 10
    cdef double learning_rate = 1e-4
    cdef double regularization = 1e-5
    
    cdef int[:] URM_train_coo_row = URM_train_coo.row
    cdef int[:] URM_train_coo_col = URM_train_coo.col
    cdef double[:] URM_train_coo_data = URM_train_coo.data
    
    cdef double[:,:] user_factors = np.random.random((n_users, num_factors))
    cdef double[:,:] item_factors = np.random.random((n_items, num_factors))
    cdef double H_i, W_u
    cdef double item_factors_update, user_factors_update
                
    cdef double loss = 0.0
    cdef long start_time = time.time()

    for sample_num in range(URM_train.nnz):

        # Randomly pick sample
        sample_index = rand() % n_interactions

        user_id = URM_train_coo_row[sample_index]
        item_id = URM_train_coo_col[sample_index]
        rating = URM_train_coo_data[sample_index]

        # Compute prediction
        predicted_rating = 0.0
        
        for factor_index in range(num_factors):
            predicted_rating += user_factors[user_id, factor_index] * item_factors[item_id, factor_index]
 
        # Compute prediction error, or gradient
        prediction_error = rating - predicted_rating
        loss += prediction_error**2

        # Copy original value to avoid messing up the updates
        for factor_index in range(num_factors):
            
            H_i = item_factors[item_id,factor_index]
            W_u = user_factors[user_id,factor_index]  

            user_factors_update = prediction_error * H_i - regularization * W_u
            item_factors_update = prediction_error * W_u - regularization * H_i

            user_factors[user_id,factor_index] += learning_rate * user_factors_update 
            item_factors[item_id,factor_index] += learning_rate * item_factors_update    

        # Print some stats
        if (sample_num +1)% 500000 == 0:
            elapsed_time = time.time() - start_time
            samples_per_second = sample_num/elapsed_time
            print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

    return loss, samples_per_second

In [30]:
loss, samples_per_second = do_some_training(URM_train)

Iteration 500000 in 0.58 seconds, loss is 1.92. Samples per second 862352.01
Iteration 1000000 in 0.67 seconds, loss is 1.65. Samples per second 1501920.70
Iteration 1500000 in 0.75 seconds, loss is 1.50. Samples per second 1989868.19
Iteration 2000000 in 0.84 seconds, loss is 1.40. Samples per second 2375793.52
Iteration 2500000 in 0.93 seconds, loss is 1.32. Samples per second 2691561.53
Iteration 3000000 in 1.02 seconds, loss is 1.26. Samples per second 2953239.25
Iteration 3500000 in 1.10 seconds, loss is 1.22. Samples per second 3167883.28
Iteration 4000000 in 1.19 seconds, loss is 1.18. Samples per second 3353331.10
Iteration 4500000 in 1.28 seconds, loss is 1.14. Samples per second 3513294.63
Iteration 5000000 in 1.37 seconds, loss is 1.12. Samples per second 3658035.84
Iteration 5500000 in 1.45 seconds, loss is 1.09. Samples per second 3780435.58
Iteration 6000000 in 1.55 seconds, loss is 1.07. Samples per second 3881324.90
Iteration 6500000 in 1.63 seconds, loss is 1.05. Sampl

In [31]:
estimated_seconds = 8e6 * 10 / samples_per_second
print("Estimated time with the previous training speed is {:.2f} seconds, or {:.2f} minutes".format(estimated_seconds, estimated_seconds/60))

Estimated time with the previous training speed is 18.98 seconds, or 0.32 minutes


### Works nicely, let's put an additional for loop to do multiple epochs

In [34]:
%%cython
import numpy as np
import time

from libc.stdlib cimport rand, srand, RAND_MAX

def train_multiple_epochs(URM_train, learning_rate_input, n_epochs):
    
    URM_train_coo = URM_train.tocoo()
    n_users, n_items = URM_train_coo.shape
    cdef int n_interactions = URM_train.nnz
    
    cdef int sample_num, sample_index, user_id, item_id, factor_index
    cdef double rating, predicted_rating, prediction_error

    cdef int num_factors = 10
    cdef double learning_rate = 1e-4
    cdef double regularization = 1e-5
    
    cdef int[:] URM_train_coo_row = URM_train_coo.row
    cdef int[:] URM_train_coo_col = URM_train_coo.col
    cdef double[:] URM_train_coo_data = URM_train_coo.data

    cdef double[:,:] user_factors = np.random.random((n_users, num_factors))
    cdef double[:,:] item_factors = np.random.random((n_items, num_factors))
    cdef double H_i, W_u
    cdef double item_factors_update, user_factors_update
                
    cdef double loss = 0.0
    cdef long start_time = time.time()
    
    for n_epoch in range(n_epochs):

        loss = 0.0
        start_time = time.time()

        for sample_num in range(URM_train.nnz):

            # Randomly pick sample
            sample_index = rand() % n_interactions

            user_id = URM_train_coo_row[sample_index]
            item_id = URM_train_coo_col[sample_index]
            rating = URM_train_coo_data[sample_index]

            # Compute prediction
            predicted_rating = 0.0

            for factor_index in range(num_factors):
                predicted_rating += user_factors[user_id, factor_index] * item_factors[item_id, factor_index]

            # Compute prediction error, or gradient
            prediction_error = rating - predicted_rating
            loss += prediction_error**2

            # Copy original value to avoid messing up the updates
            for factor_index in range(num_factors):

                H_i = item_factors[item_id,factor_index]
                W_u = user_factors[user_id,factor_index]  

                user_factors_update = prediction_error * H_i - regularization * W_u
                item_factors_update = prediction_error * W_u - regularization * H_i

                user_factors[user_id,factor_index] += learning_rate * user_factors_update 
                item_factors[item_id,factor_index] += learning_rate * item_factors_update    

#             # Print some stats
#             if (sample_num +1)% 1000000 == 0:
#                 elapsed_time = time.time() - start_time
#                 samples_per_second = sample_num/elapsed_time
#                 print("Iteration {} in {:.2f} seconds, loss is {:.2f}. Samples per second {:.2f}".format(sample_num+1, elapsed_time, loss/sample_num, samples_per_second))

            
        elapsed_time = time.time() - start_time
        samples_per_second = sample_num/elapsed_time
     
        print("Epoch {} complete in in {:.2f} seconds, loss is {:.3E}. Samples per second {:.2f}".format(n_epoch+1, time.time() - start_time, loss/sample_num, samples_per_second))

    return np.array(user_factors), np.array(item_factors), loss, samples_per_second    

In [33]:
n_items = URM_train.shape[1]
learning_rate = 1e-3
    
user_factors, item_factors, loss, samples_per_second = \
    train_multiple_epochs(URM_train, learning_rate, 10)

Epoch 1 complete in in 1.44 seconds, loss is 9.996E-01. Samples per second 5540332.54
Epoch 2 complete in in 1.86 seconds, loss is 7.063E-01. Samples per second 4300998.79
Epoch 3 complete in in 2.28 seconds, loss is 6.368E-01. Samples per second 3510142.54
Epoch 4 complete in in 1.70 seconds, loss is 5.924E-01. Samples per second 4699807.44
Epoch 5 complete in in 2.12 seconds, loss is 5.539E-01. Samples per second 3773097.43
Epoch 6 complete in in 1.54 seconds, loss is 5.182E-01. Samples per second 5200338.93
Epoch 7 complete in in 1.96 seconds, loss is 4.830E-01. Samples per second 4080720.42
Epoch 8 complete in in 2.38 seconds, loss is 4.497E-01. Samples per second 3364858.22
Epoch 9 complete in in 1.80 seconds, loss is 4.189E-01. Samples per second 4452850.38
Epoch 10 complete in in 2.21 seconds, loss is 3.913E-01. Samples per second 3625358.64


### From 20 minutes of training time to a few seconds...